In [1]:
from random import sample, choices

In [2]:
armor_options = ['STR', 'DEX', 'LUK', 'INT', ['STR', 'DEX'], ['STR', 'INT'], ['STR', 'LUK'], \
                 ['DEX', 'INT'], ['DEX', 'LUK'], ['INT', 'LUK'], '올스탯', \
                '최대 HP', '최대 MP', '착용 레벨 감소', '방어력', '공격력', '마력', '이동속도', '점프력']

armor_divisions = {0: ['STR', 'DEX', 'LUK', 'INT', '방어력'],
                   1: [['STR', 'DEX'], ['STR', 'INT'], ['STR', 'LUK'], ['DEX', 'INT'], ['DEX', 'LUK'], ['INT', 'LUK']],
                   2: ['최대 HP', '최대 MP'],
                   3: ['착용 레벨 감소'],
                   4: ['공격력', '마력', '올스탯', '이동속도', '점프력']}

In [3]:
def Score_Simulator(level, main_stat = 'STR', main_atkmgc = '공격력', as_score = 10, atkmgc_score = 4, item = '강환불'):
    options_indices = sample(range(19), 4)
    
    data = [3, 4, 5, 6, 7]
    if level == 150:
        data = [[8* grade for grade in data],
                [4*grade for grade in data]]
    elif level == 160:
        data = [[9* grade for grade in data],
                [5*grade for grade in data]]
    elif level == 200:
        data = [[11* grade for grade in data],
                [6*grade for grade in data]]

    options = [armor_options[i] for i in options_indices]
    data += [[3*level*grade for grade in [3, 4, 5, 6, 7]],
             [-15, -20, -25, -30, -35],
             [3, 4, 5, 6, 7]]
    
    if item == '강환불':
        probs = [0.2, 0.3, 0.36, 0.14, 0]
    elif item == '영환불':
        probs = [0, 0.29, 0.45, 0.25, 0.01]
    
    options_kinds = [(options[i], key) for key, value in armor_divisions.items() for i in range(4) if options[i] in value]
    final_result = dict()
    final_result[main_stat] = 0; final_result[main_atkmgc] = 0; final_result['올스탯'] = 0
    
    for i in range(4):
        option_name = options_kinds[i][0]; option_kind = options_kinds[i][1]
        options_grade = choices(population = [0, 1, 2, 3, 4], weights = probs, k = 1)[0]
        option = data[option_kind][options_grade]
        
        if type(option_name) is list:
            for j in range(2):
                temp_option_name = option_name[j]
                if temp_option_name in final_result.keys():
                    final_result[temp_option_name] += option
                else:
                    final_result[temp_option_name] = option
        else:
            if option_name in final_result.keys():
                final_result[option_name] += option
            else:
                final_result[option_name] = option
    

    final_score = final_result[main_stat] + as_score * final_result['올스탯'] + atkmgc_score * final_result[main_atkmgc]
    
    return final_score

In [4]:
def score_proportion(standard_score, level = 200, as_score = 10.125, atkmgc_score = 5.45, item = '영환불', iterations = 10000):
    result = (Score_Simulator(level = level, as_score = as_score, atkmgc_score = atkmgc_score, item = item) for i in range(iterations))
    scores = [1 for score in result if score >= standard_score]
    print('Proportion of Cases that satisfied the standard is {:.3f}% (total {})'.format(sum(scores) / iterations * 100, sum(scores)))
    
    return result

In [5]:
level = 160
score_proportion(standard_score = 100, level = level, item = '강환불', iterations = 100000)
score_proportion(standard_score = 110, level = level, item = '강환불', iterations = 100000)
score_proportion(standard_score = 120, level = level, item = '강환불', iterations = 100000)
score_proportion(standard_score = 130, level = level, item = '강환불', iterations = 100000)
None

Proportion of Cases that satisfied the standard is 2.186% (total 2186)
Proportion of Cases that satisfied the standard is 1.066% (total 1066)
Proportion of Cases that satisfied the standard is 0.474% (total 474)
Proportion of Cases that satisfied the standard is 0.180% (total 180)


In [6]:
level = 160
score_proportion(standard_score = 100, level = level, iterations = 100000)
score_proportion(standard_score = 110, level = level, iterations = 100000)
score_proportion(standard_score = 120, level = level, iterations = 100000)
score_proportion(standard_score = 130, level = level, iterations = 100000)
None

Proportion of Cases that satisfied the standard is 4.141% (total 4141)
Proportion of Cases that satisfied the standard is 2.111% (total 2111)
Proportion of Cases that satisfied the standard is 1.087% (total 1087)
Proportion of Cases that satisfied the standard is 0.468% (total 468)


In [7]:
level = 200
score_proportion(standard_score = 100, level = level, item = '강환불', iterations = 100000)
score_proportion(standard_score = 110, level = level, item = '강환불', iterations = 100000)
score_proportion(standard_score = 120, level = level, item = '강환불', iterations = 100000)
score_proportion(standard_score = 130, level = level, item = '강환불', iterations = 100000)
None

Proportion of Cases that satisfied the standard is 4.221% (total 4221)
Proportion of Cases that satisfied the standard is 2.349% (total 2349)
Proportion of Cases that satisfied the standard is 1.266% (total 1266)
Proportion of Cases that satisfied the standard is 0.577% (total 577)


In [8]:
level = 200
score_proportion(standard_score = 100, level = level, iterations = 100000)
score_proportion(standard_score = 110, level = level, iterations = 100000)
score_proportion(standard_score = 120, level = level, iterations = 100000)
score_proportion(standard_score = 130, level = level, iterations = 100000)
None

Proportion of Cases that satisfied the standard is 7.139% (total 7139)
Proportion of Cases that satisfied the standard is 4.456% (total 4456)
Proportion of Cases that satisfied the standard is 2.565% (total 2565)
Proportion of Cases that satisfied the standard is 1.361% (total 1361)
